In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings  # Updated import
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.storage import LocalFileStore


llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./files/moby_dick.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Correct the method call and argument
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,  # Assuming 'embeddings' is your Embeddings object
    document_embedding_cache=cache_dir,  # Assuming 'cache_dir' is your ByteStore object
)

vectorstore = FAISS.from_documents(documents=docs, embedding=cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about the text. if you don't know the answer, just say that you don't know.:\n\n{context}"),
    ("human", "{question}"),
])

chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm


chain.invoke("Who is the main character of the story?")

AIMessage(content='The main character of the story is Ishmael, who serves as the narrator of "Moby-Dick."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 1763, 'total_tokens': 1785, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-d709aefb-a4cb-40b8-a03e-f6eb9016f2da-0', usage_metadata={'input_tokens': 1763, 'output_tokens': 22, 'total_tokens': 1785, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})